## Import library

In [55]:
import pandas as pd
import httpx
import json
import gzip
import string
import datetime
import random

## Set option pandas

In [56]:
pd.options.mode.use_inf_as_na = True
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Read configuration

In [57]:
with open("./credentials.json") as r:
    config = json.loads(r.read())
minio_config = dict(
    key=config["accessKey"],
    secret=config["secretKey"],
    client_kwargs={
        'endpoint_url': config["url"]
    }
)

with open("./api_config.json") as r:
    config = json.loads(r.read())
api_config = dict(
    url=config["url"],
    token=config["token"],
)

## Function call api

In [58]:
def get_process(httpx, url, headers, topic):
  url = url + f"?topic={topic}"
  with httpx.Client() as client:
    r = client.get(url=url, headers=headers)
    json = r.json()
  if json["ok"] == True and len(json["rows"]) != 0:
    return json["rows"]
  else:
    return {}


def set_process(httpx, url, headers, process_id, status):
  url = url + "/status"
  headers["Content-Type"] = "application/x-www-form-urlencoded"
  data = f"processId={process_id}&status={status}"
  with httpx.Client() as client:
    r = client.put(url=url, headers=headers, data=data)
    json = r.json()
  if (json["ok"] == True):
    return json
  else:
    return {}

## Get process from api

In [59]:
# topic = "service_opd_screening"
topic = "appointment"
url = api_config["url"]
token = api_config["token"]
headers = {"Authorization": "Bearer " + token}
r = get_process(httpx=httpx, url=url, headers=headers, topic=topic)

display(r)

{'process_id': '258d59e1-78c5-4aea-be8c-164c5489fb5b',
 'hospcode': '41124',
 'status': 'PROCESS',
 'topic': 'appointment',
 'path': 'upload/41124/appointment/258d59e1-78c5-4aea-be8c-164c5489fb5b'}

## Set parameter after call api

In [60]:
hospcode = r["hospcode"]
process_id = r["process_id"]
source_path = "s3://mophgw/{}".format(r["path"])
destination_path = "s3://phr/{}".format(r["path"])
local_code_path = "s3://mapcode/{}/localcode".format(hospcode)
std_code_path = "s3://mapcode/{}/stdcode".format(hospcode)

## function to read_csv


In [61]:
def read_csv(path: str = "", filename: str = "", is_prefix: bool = True, prefix_type: str = ""):
  df = pd.read_csv(f"{path}/{filename}", engine="c",
              dtype="str", storage_options=minio_config)
  if is_prefix:
    prefix = filename.removesuffix(".csv.gz")
    if prefix_type == "local":
      df = df.add_prefix(f"_{prefix}_local")
    elif prefix_type == "std":
      df = df.add_prefix(f"_{prefix}_")
  return df

## load transaction file to dataframe
- person

In [62]:
person = read_csv(path=source_path, filename="person.csv.gz",is_prefix=False)

## load localcode files to dataframe
- title, gender, rh_blood_group, blood_group, nationality, marital_status, occupation

In [63]:
local_config = ["blood_group", "gender", "marital_status",
                "nationality", "rh_blood_group", "title"]
local_df = dict()
for k in local_config:
  local_df[k] = read_csv(path=local_code_path, filename=k +
                         ".csv.gz", is_prefix=True, prefix_type="local")
  display(local_df[k])

,_blood_group_localcode,_blood_group_localname


,_gender_localcode,_gender_localname
0,2,หญิง
1,1,ชาย


,_marital_status_localcode,_marital_status_localname


,_nationality_localcode,_nationality_localname
0,95,"ไทย-จีน, จีน-ไทย(ยกเลิก)"
1,96,ไร้สัญชาติ
2,97,อื่น ๆ
3,94,"ไทย-อิสลาม, อิสลาม-ไทย(ยกเลิก)"
4,91,ผู้อพยพอินโดจีนสัญชาติกัมพูชา
...,...,...
102,81,ลาว(แรงงาน)
103,82,เซอร์เบียน
104,83,มอนเตเนกริน
105,989,บุคคลที่ไม่มีสถานะทางทะเบียน


,_rh_blood_group_localcode,_rh_blood_group_localname


,_title_localcode,_title_localname
0,Ms.,Ms.
1,Mrs.,Mrs.
2,Mr.,Mr.
3,Miss.,Miss.
4,อาสาสมัครทหารพราน,อาสาสมัครทหารพราน
...,...,...
200,จ.ท.,จ.ท.
201,จ.ต.หญิง,จ.ต.หญิง
202,จ.ต.,จ.ต.
203,คุณหญิง,คุณหญิง


## load stdcode file to dataframe
- title, gender, rh_blood_group, blood_group, nationality, marital_status, occupation

In [64]:
# std_config = ["blood_group", "gender", "marital_status",
#               "nationality", "rh_blood_group", "title"]
std_config = dict(
    blood_group=["blood_group", "blood_groups"],
    gender = ["gender","genders"], 
    marital_status = ["marital_status","marital_statuses"],
    nationality = ["nationality","nationalities"], 
    rh_blood_group = ["rh_blood_group","rh_blood_groups"], 
    title = ["title","titles"]
)
std_df = dict()
for k, v in std_config.items():
  std_df[k] = read_csv(path=std_code_path, filename=v[1] + ".csv.gz", is_prefix=True, prefix_type="std")
  display(std_df[k])

,_blood_groups_localcode,_blood_groups_stdcode,_blood_groups_stdname_th,_blood_groups_stdname_en


,_genders_localcode,_genders_stdcode,_genders_stdname_th,_genders_stdname_en
0,1,1,ชาย,Male
1,2,2,หญิง,Female
2,NaN,0,ไม่สามารถระบุได้,cannot be identified


,_marital_statuses_localcode,_marital_statuses_stdcode,_marital_statuses_stdname_th,_marital_statuses_stdname_en
0,1,1,โสด,Single
1,2,2,สมรส,Married
2,3,3,หม้าย,Widowed
3,4,4,หย่า,Divorced
4,5,5,แยกกันอยู่,Separated
...,...,...,...,...
774,NaN,9005,ลูกจ้างชั่วคราว,NaN
775,NaN,9006,เจ้าพนักงานทันตสาธารณสุข,NaN
776,NaN,9007,เจ้าพนักงานสาธารณสุข,NaN
777,NaN,9998,ไม่ทราบ / ไม่ระบุ,NaN


,_nationalities_localcode,_nationalities_stdcode,_nationalities_stdname_th,_nationalities_stdname_en
0,02,2,โปรตุเกส,PORTUGUESE
1,03,3,ดัตช์,NETHERLANDS
2,04,4,เยอรมัน,W. GERMANY
3,05,5,ฝรั่งเศส,FRANCE
4,06,6,เดนมาร์ก,DENMARK
...,...,...,...,...
499,NaN,NaN,NaN,NaN
500,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN
502,NaN,NaN,NaN,NaN


,_rh_blood_groups_localcode,_rh_blood_groups_stdcode,_rh_blood_groups_stdname_th,_rh_blood_groups_stdname_en


,_titles_localcode,_titles_stdcode,_titles_stdname_th,_titles_stdname_en
0,NaN,001,เด็กชาย,NaN
1,NaN,002,เด็กหญิง,NaN
2,NaN,003,นาย,MR.
3,NaN,004,นางสาว,MISS
4,NaN,005,นาง,MRS.
...,...,...,...,...
499,NaN,956,พระครูสมุทรขันตยาภรณ์,NaN
500,NaN,957,พระครูภาวนาวรกิจ,NaN
501,NaN,958,พระครูศรีศาสนคุณ,NaN
502,NaN,959,พระครูวิบูลย์ธรรมศาสก์,NaN


## merge localcode to person

In [65]:
person_merge_local = dict(
  title = ["title_code","_title_localcode"],
  gender = ["gender","_gender_localcode"],
  rh_blood_group = ["rh_blood_group", "_rh_blood_group_localcode"],
  blood_group = ["blood_group","_blood_group_localcode"],
  nationality = ["nationality_code","_nationality_localcode"],
  marital_status = ["marital_status","_marital_status_localcode"],
)
for k, v in person_merge_local.items():
  person = pd.merge(
      person, local_df[k], how="left", left_on=v[0], right_on=v[1])

display(person)

,cid,hn,title_code,first_name,middle_name,last_name,birthdate,gender,rh_blood_group,blood_group,nationality_code,marital_status,_title_localcode,_title_localname,_gender_localcode,_gender_localname,_rh_blood_group_localcode,_rh_blood_group_localname,_blood_group_localcode,_blood_group_localname,_nationality_localcode,_nationality_localname,_marital_status_localcode,_marital_status_localname
0,3310401305351,000480857,นาย,ไล,NaN,วิเศษดี,1950-09-20T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,1,นาย,นาย,1,ชาย,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
1,1104700315285,000436214,ด.ช.,กวีวัฒน์,NaN,กัลยา,2015-05-07T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,1,ด.ช.,ด.ช.,1,ชาย,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
2,3160500223771,000023032,นาง,สมหญิง,NaN,เต็งประวัติ,1959-07-29T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
3,3601000069582,000092089,นาง,นวลฉวี,NaN,กลีบผะกา,1961-06-03T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
4,1319800282098,000123199,น.ส.,ศิริกาญจน์,NaN,วอนเจียม,2000-04-01T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,น.ส.,น.ส.,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1319800909571,000574373,ด.ช.,ปุญญพัฒน์,NaN,สวัสดี,2022-05-26T17:00:00.000Z,1,NaN,NaN,99,1,ด.ช.,ด.ช.,1,ชาย,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
75,3310400832765,000052245,น.ส.,ประทีป,NaN,ดาลาด,1976-05-20T17:00:00.000Z,2,NaN,5,99,2,น.ส.,น.ส.,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
76,3310400853118,000018076,นาง,ละม่อม,NaN,กลิ่นบุหงา,1966-12-21T17:00:00.000Z,2,NaN,ไม่ทราบหมู่เลือด,99,2,นาง,นาง,2,หญิง,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN
77,3311200168196,000426804,นาย,อุระ,NaN,สุนทรินทร์,1950-12-14T17:00:00.000Z,1,NaN,ไม่ทราบหมู่เลือด,99,2,นาย,นาย,1,ชาย,NaN,NaN,NaN,NaN,99,ไทย,NaN,NaN


## merge stdcode to person

In [66]:
person_merge_std = dict(
    title=["_title_localcode", "_titles_localcode"],
    gender=["_gender_localcode", "_genders_localcode"],
    rh_blood_group=["_rh_blood_group_localcode", "_rh_blood_groups_localcode"],
    blood_group=["_blood_group_localcode", "_blood_groups_localcode"],
    nationality=["_nationality_localcode", "_nationalities_localcode"],
    marital_status=["_marital_status_localcode", "_marital_statuses_localcode"],
)
for k, v in person_merge_std.items():
    person = pd.merge(person, std_df[k], how="left", left_on=f"{v[0]}", right_on=f"{v[1]}")
    # person = person.drop_duplicates(subset=["cid"])
display(person.columns)

Index(['cid', 'hn', 'title_code', 'first_name', 'middle_name', 'last_name', 'birthdate', 'gender', 'rh_blood_group', 'blood_group', 'nationality_code', 'marital_status', '_title_localcode', '_title_localname', '_gender_localcode', '_gender_localname', '_rh_blood_group_localcode', '_rh_blood_group_localname', '_blood_group_localcode', '_blood_group_localname', '_nationality_localcode', '_nationality_localname', '_marital_status_localcode', '_marital_status_localname', '_titles_localcode', '_titles_stdcode', '_titles_stdname_th', '_titles_stdname_en', '_genders_localcode', '_genders_stdcode', '_genders_stdname_th', '_genders_stdname_en', '_rh_blood_groups_localcode', '_rh_blood_groups_stdcode', '_rh_blood_groups_stdname_th', '_rh_blood_groups_stdname_en', '_blood_groups_localcode', '_blood_groups_stdcode', '_blood_groups_stdname_th', '_blood_groups_stdname_en', '_nationalities_localcode', '_nationalities_stdcode', '_nationalities_stdname_th', '_nationalities_stdname_en',
       '_marital

## rename columns

In [67]:
# person = person[[
#     'cid', 'hn', 
#     'title_code', '_title_localcode', '_title_localname', '_titles_localcode', '_titles_stdcode', '_titles_stdname_th', '_titles_stdname_en',
#     'first_name', 'middle_name', 'last_name', 'birthdate', 
#     'gender', '_gender_localcode', '_gender_localname', '_genders_localcode', '_genders_stdcode', '_genders_stdname_th', '_genders_stdname_en',
#     'rh_blood_group', '_rh_blood_group_localcode', '_rh_blood_group_localname', '_rh_blood_groups_localcode', '_rh_blood_groups_stdcode', '_rh_blood_groups_stdname_th', '_rh_blood_groups_stdname_en',
#     'blood_group', '_blood_group_localcode', '_blood_group_localname', '_blood_groups_localcode', '_blood_groups_stdcode', '_blood_groups_stdname_th', '_blood_groups_stdname_en',
#     'nationality_code', '_nationality_localcode', '_nationality_localname', '_nationalities_localcode', '_nationalities_stdcode', '_nationalities_stdname_th', '_nationalities_stdname_en',
#     'marital_status', '_marital_status_localcode', '_marital_status_localname', '_marital_statuses_localcode', '_marital_statuses_stdcode', '_marital_statuses_stdname_th', '_marital_statuses_stdname_en'
# ]]
person = person[[
    'cid', 'hn', 
    '_title_localcode', '_title_localname', '_titles_stdcode', '_titles_stdname_th', '_titles_stdname_en',
    'first_name', 'middle_name', 'last_name', 'birthdate', 
    '_gender_localcode', '_gender_localname', '_genders_stdcode', '_genders_stdname_th', '_genders_stdname_en',
    '_rh_blood_group_localcode', '_rh_blood_group_localname', '_rh_blood_groups_stdcode', '_rh_blood_groups_stdname_th', '_rh_blood_groups_stdname_en',
    '_blood_group_localcode', '_blood_group_localname', '_blood_groups_stdcode', '_blood_groups_stdname_th', '_blood_groups_stdname_en',
    '_nationality_localcode', '_nationality_localname', '_nationalities_stdcode', '_nationalities_stdname_th', '_nationalities_stdname_en',
    '_marital_status_localcode', '_marital_status_localname', '_marital_statuses_stdcode', '_marital_statuses_stdname_th', '_marital_statuses_stdname_en'
]]
person.columns = [
    'cid', 'hn', 
    '_title_localcode', '_title_localname', '_titles_stdcode', '_titles_stdname_th', '_titles_stdname_en',
    'first_name', 'middle_name', 'last_name', 'birthdate', 
    '_gender_localcode', '_gender_localname', '_gender_stdcode', '_gender_stdname_th', '_gender_stdname_en',
    '_rh_blood_group_localcode', '_rh_blood_group_localname', '_rh_blood_group_stdcode', '_rh_blood_group_stdname_th', '_rh_blood_group_stdname_en',
    '_blood_group_localcode', '_blood_group_localname', '_blood_group_stdcode', '_blood_group_stdname_th', '_blood_group_stdname_en',
    '_nationality_localcode', '_nationality_localname', '_nationality_stdcode', '_nationality_stdname_th', '_nationality_stdname_en',
    '_marital_status_localcode', '_marital_status_localname', '_marital_status_stdcode', '_marital_status_stdname_th', '_marital_status_stdname_en'
]
display(person.columns)

Index(['cid', 'hn', '_title_localcode', '_title_localname', '_titles_stdcode', '_titles_stdname_th', '_titles_stdname_en', 'first_name', 'middle_name', 'last_name', 'birthdate', '_gender_localcode', '_gender_localname', '_gender_stdcode', '_gender_stdname_th', '_gender_stdname_en', '_rh_blood_group_localcode', '_rh_blood_group_localname', '_rh_blood_group_stdcode', '_rh_blood_group_stdname_th', '_rh_blood_group_stdname_en', '_blood_group_localcode', '_blood_group_localname', '_blood_group_stdcode', '_blood_group_stdname_th', '_blood_group_stdname_en', '_nationality_localcode', '_nationality_localname', '_nationality_stdcode', '_nationality_stdname_th', '_nationality_stdname_en', '_marital_status_localcode', '_marital_status_localname', '_marital_status_stdcode', '_marital_status_stdname_th', '_marital_status_stdname_en'], dtype='object')

In [68]:
# with open("./person_config.json", "w") as fw:
#   fw.write(json.dumps(dict(std=person_merge_std, local=person_merge_local)))